In [ ]:
import yfinance as yf
import pandas as pd
import random
from datetime import datetime, timedelta

def fetch_random_slices(symbol, frontier_date, ending_date, 
                        interval="1d", slice_length=timedelta(days=1), n_slices=5):
    """
    Picks independent slices of stock movement using Yahoo Finance data.
    
    Parameters
    ----------
    symbol : str
        Ticker symbol (e.g., "AAPL").
    frontier_date : datetime
        Earliest possible start date.
    ending_date : datetime
        Latest possible end date.
    interval : str
        Data granularity (e.g., "1m", "5m", "1h", "1d").
    slice_length : timedelta
        How long each slice should be (e.g., 1 day, 3 hours).
    n_slices : int
        Number of random slices to return.
    
    Returns
    -------
    list of dicts { "start_date", "start_price", "end_date", "end_price" }
    """

    # --- Download price history ---
    df = yf.download(
        symbol,
        start=frontier_date.strftime("%Y-%m-%d"),
        end=ending_date.strftime("%Y-%m-%d"),
        interval=interval,
        progress=False
    )
    
    if df.empty:
        return []
    
    df = df.reset_index()  # Ensure 'Datetime' is a column
    if "Datetime" in df.columns:
        time_col = "Datetime"
    else:
        time_col = "Date"

    results = []
    for _ in range(n_slices):
        latest_start = ending_date - slice_length
        if frontier_date >= latest_start:
            break  # Not enough room for slices
        
        rand_start = frontier_date + timedelta(
            seconds=random.randint(0, int((latest_start - frontier_date).total_seconds()))
        )
        rand_end = rand_start + slice_length

        start_row = df.loc[df[time_col] >= rand_start].head(1)
        end_row   = df.loc[df[time_col] >= rand_end].head(1)
        
        if not start_row.empty and not end_row.empty:
            results.append({
                "start_date": start_row[time_col].iloc[0],
                "start_price": float(start_row["Close"].iloc[0]),
                "end_date": end_row[time_col].iloc[0],
                "end_price": float(end_row["Close"].iloc[0])
            })
    
    return results

: 

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def fetch_consecutive_slices(symbol, frontier_date, ending_date, 
                             interval="1d", slice_length=timedelta(days=1), 
                             max_slices=None):
    """
    Fetches consecutive, non-overlapping slices of stock movement using Yahoo Finance data.
    Each slice is frontier_date → frontier_date+slice_length, then continues until ending_date.
    
    Parameters
    ----------
    symbol : str
        Ticker symbol (e.g., "AAPL").
    frontier_date : datetime
        Earliest possible start date.
    ending_date : datetime
        Latest possible end date.
    interval : str
        Data granularity (e.g., "1m", "5m", "1h", "1d").
    slice_length : timedelta
        Length of each slice (e.g., 1 day, 3 hours).
    max_slices : int, optional
        Maximum number of slices to return. Default = all possible slices.
    
    Returns
    -------
    list of dicts { "start_date", "start_price", "end_date", "end_price" }
    """

    # --- Download full history once ---
    df = yf.download(
        symbol,
        start=frontier_date.strftime("%Y-%m-%d"),
        end=ending_date.strftime("%Y-%m-%d"),
        interval=interval,
        progress=False
    )
    
    if df.empty:
        return []
    
    df = df.reset_index()
    if "Datetime" in df.columns:
        time_col = "Datetime"
    else:
        time_col = "Date"

    results = []
    current_start = frontier_date
    n_slices = 0

    while current_start + slice_length <= ending_date:
        current_end = current_start + slice_length

        # Find nearest available prices
        start_row = df.loc[df[time_col] >= current_start].head(1)
        end_row   = df.loc[df[time_col] >= current_end].head(1)

        if not start_row.empty and not end_row.empty:
            results.append({
                "start_date": start_row[time_col].iloc[0],
                "start_price": float(start_row["Close"].iloc[0]),
                "end_date": end_row[time_col].iloc[0],
                "end_price": float(end_row["Close"].iloc[0])
            })
            n_slices += 1

        # Stop if we’ve hit the slice cap
        if max_slices is not None and n_slices >= max_slices:
            break

        # Move to next slice
        current_start = current_end

    return results

In [ ]:
import pandas as pd
import numpy as np

def slices_to_extended_dataframe(slices, time_format="%Y-%m-%d|%H:%M"):
    """
    Converts slice outputs into an extended DataFrame with:
    - linked date ranges
    - raw price change
    - % change
    - log returns (%)
    - cumulative % return (compounded)
    - cumulative log return (%)
    - cumulative return from logs (%)
    
    Parameters
    ----------
    slices : list of dicts
        Output from fetch_random_slices or fetch_consecutive_slices.
        Each slice dict must have keys: 
            start_date, end_date, start_price, end_price
    time_format : str
        How to format the timestamps in the linked range.
    
    Returns
    -------
    pd.DataFrame
        Columns: 
        [date_range, start_date, end_date, raw_change, pct_change, 
         log_return, cum_return, cum_log_return, cum_return_from_logs]
    """
    if not slices:
        return pd.DataFrame(columns=[
            "date_range", "start_date", "end_date", 
            "raw_change", "pct_change", "log_return", 
            "cum_return", "cum_log_return", "cum_return_from_logs"
        ])
    
    rows = []
    for s in slices:
        start_date = pd.to_datetime(s["start_date"])
        end_date   = pd.to_datetime(s["end_date"])
        start_price = float(s["start_price"])
        end_price   = float(s["end_price"])
        
        # Format date range string
        start_str = start_date.strftime(time_format)
        end_str   = end_date.strftime(time_format)
        date_range = f"{start_str} : {end_str}"
        
        # Compute changes
        raw_change = end_price - start_price
        pct_change = (raw_change / start_price) * 100
        log_return = np.log(end_price / start_price) * 100  # %
        
        rows.append({
            "date_range": date_range,
            "start_date": start_date,
            "end_date": end_date,
            "raw_change": raw_change,
            "pct_change": pct_change,
            "log_return": log_return
        })
    
    df = pd.DataFrame(rows)
    
    # Cumulative compounded % return
    df["cum_return"] = (1 + df["pct_change"]/100).cumprod() - 1
    df["cum_return"] = df["cum_return"] * 100  # %
    
    # Cumulative additive log return (in %)
    df["cum_log_return"] = df["log_return"].cumsum()
    
    # Convert cumulative log return into implied compounded return (%)
    df["cum_return_from_logs"] = (np.exp(df["cum_log_return"]/100) - 1) * 100
    
    return df

In [ ]:
import pandas as pd

def slices_to_dataframe(slices):
    """
    Converts slice output into a DataFrame with raw % change.
    
    Parameters
    ----------
    slices : list of dicts
        Output from fetch_random_slices or fetch_consecutive_slices.
        Each dict must have: start_date, end_date, start_price, end_price.
    
    Returns
    -------
    pd.DataFrame
        Columns: [start_date, end_date, start_price, end_price, pct_change]
    """
    if not slices:
        return pd.DataFrame(columns=[
            "start_date", "end_date", "start_price", "end_price", "pct_change"
        ])
    
    df = pd.DataFrame(slices)
    # Ensure numeric type for calculations
    df["start_price"] = pd.to_numeric(df["start_price"], errors="coerce")
    df["end_price"]   = pd.to_numeric(df["end_price"], errors="coerce")
    
    df["pct_change"] = (df["end_price"] - df["start_price"]) / df["start_price"] * 100
    return df

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# --- Parameters ---
symbol = "AAPL"
frontier_date = datetime(2022, 1, 1)
ending_date   = datetime(2022, 3, 1)

# --- Consecutive slices ---
cons_slices = fetch_consecutive_slices(
    symbol,
    frontier_date=frontier_date,
    ending_date=ending_date,
    interval="1d",
    slice_length=timedelta(days=5),
    max_slices=4
)

print("\nConsecutive slices (raw dicts):")
for s in cons_slices:
    print(s)

cons_df = slices_to_dataframe(cons_slices)
print("\nConsecutive slices → DataFrame:")
print(cons_df)

cons_ext_df = slices_to_extended_dataframe(cons_slices)
print("\nConsecutive slices → Extended DataFrame:")
print(cons_ext_df)

# --- Random slices ---
rand_slices = fetch_random_slices(
    symbol,
    frontier_date=frontier_date,
    ending_date=ending_date,
    interval="1d",
    slice_length=timedelta(days=5),
    n_slices=4
)

print("\nRandom slices (raw dicts):")
for s in rand_slices:
    print(s)

rand_df = slices_to_dataframe(rand_slices)
print("\nRandom slices → DataFrame:")
print(rand_df)

rand_ext_df = slices_to_extended_dataframe(rand_slices)
print("\nRandom slices → Extended DataFrame:")
print(rand_ext_df)